# PRE-PROCESSING DATA
Mengimport library :
- nltk
- tensorflow
- numpy
- matplotlib (for graph)

In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation 
from tensorflow.keras.optimizers import SGD
import random
import pandas as pd
import matplotlib.pyplot as plt

# Membuka Intents sebagai Bank Kata Yang Dipakai
berformat json, memiliki isi tag dan response chatbot

In [5]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = pd.read_json('/content/intents.json')

In [6]:
data_file.head()

,intents
0,"{'tag': 'Ucapan Selamat Datang', 'patterns': [..."
1,"{'tag': 'Kalimat Aduan', 'patterns': ['Saya in..."
2,"{'tag': 'Ucapan Terima kasih', 'patterns': ['T..."
3,"{'tag': 'Jam buka', 'patterns': ['Jam berapa b..."
4,"{'tag': 'pembayaran', 'patterns': ['Bagaimana ..."


# TOKENISASI KATA

In [7]:
for intent in data_file['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [8]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

34 documents
6 classes ['Buka Hari ini', 'Jam buka', 'Kalimat Aduan', 'Ucapan Selamat Datang', 'Ucapan Terima kasih', 'pembayaran']
45 unique lemmatized words ['apakah', 'bagaimana', 'baiklah', 'bantuannya', 'bayar', 'bengkel', 'berapa', 'bertanya', 'bisa', 'buka', 'cara', 'e-wallet', 'hai', 'hallo', 'hari', 'informasi', 'ingin', 'ini', 'jam', 'kasih', 'kebingungan', 'kesulitan', 'm-banking', 'malam', 'masukkannya', 'melapor', 'membantu', 'memerlukan', 'mengalami', 'menggunakan', 'metode', 'oke', 'pagi', 'pakai', 'pembayarannya', 'sampai', 'sangat', 'sarannya', 'saya', 'selamat', 'siang', 'sore', 'te', 'terima', 'terimakasih']


# TRAINING DATA
Persiapan training data dan patternisasi

In [9]:
# initializing training data
training = []
output_empty = [0] * len(classes)

for document in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = document[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Data Training Telah Tersedia")

Data Training Telah Tersedia


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


# CREATE MODEL
Menambahkan layer pada model learning yang akan dibuat

In [10]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [11]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), 
                 np.array(train_y), 
                 epochs=1000, 
                 batch_size=8, 
                 verbose=1)
model.save('chatbot_model.h5', hist)

print("model telah tersedia")

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/1000
5/5 [==============================] - 1s 6ms/step - loss: 1.7965 - accuracy: 0.1765
Epoch 2/1000
5/5 [==============================] - 0s 4ms/step - loss: 1.7451 - accuracy: 0.2059
Epoch 3/1000
5/5 [==============================] - 0s 4ms/step - loss: 1.6671 - accuracy: 0.3235
Epoch 4/1000
5/5 [==============================] - 0s 4ms/step - loss: 1.6273 - accuracy: 0.4706
Epoch 5/1000
5/5 [==============================] - 0s 3ms/step - loss: 1.5513 - accuracy: 0.4412
Epoch 6/1000
5/5 [==============================] - 0s 3ms/step - loss: 1.4860 - accuracy: 0.6765
Epoch 7/1000
5/5 [==============================] - 0s 3ms/step - loss: 1.5302 - accuracy: 0.5294
Epoch 8/1000
5/5 [==============================] - 0s 3ms/step - loss: 1.3669 - accuracy: 0.6176
Epoch 9/1000
5/5 [==============================] - 0s 3ms/step - loss: 1.3127 - accuracy: 0.5588
Epoch 10/1000
5/5 [==============================] - 0s 4ms/step - loss: 1.2811 - accuracy: 0.6176
Epoch 11/1000
5/5 [

In [12]:
hist.history.keys()

dict_keys(['loss', 'accuracy'])

In [14]:
from tensorflow.keras.models import load_model

model = load_model('chatbot_model.h5')
data_file = pd.read_json('/content/intents.json')
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

# SUMMARY & GRAPH

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               5888      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 6)                 390       
                                                                 
Total params: 14,534
Trainable params: 14,534
Non-trainable params: 0
_________________________________________________________________


# CHATBOT MODEL

In [16]:
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(w.lower()) for sentence in sentence_words]
  return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
   # tokenize the pattern
  sentence_words = clean_up_sentence(sentence)
  # bag of words - matrix of N words, vocabulary matrix
  bag = 0 * len(words)
  for s in sentence_words:
    for i,w in enumerate(words):
      if w == s:
        # assign 1 if current word is in the vocabulary position
        bag[i] = 1
        if show_details:
          print("founds in bag: %s" % w)
  return(np.array(bag))  

def predict_class(sentence, model):
  # filter out predictions below a threshold
  p = bow(sentence, words, show_details=False)
  res = model.predict(np.array([p]))[0]
  ERROR_THREESHOLD = 0.25
  result = [[i,r] for i,r in enumerate(res) if r>ERROR_THREESHOLD]
  # sort by strength of probability
  result.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for n in result:
    return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
  return return_list

def getResponse(ints, intents_json):
  tag = ints[0]['intent']
  list_of =_intents = intents_json['intents']
  for i in ints:
    if(i['tag'] == tag):
      result = random.choice(i['responses'])
      break
  return result

# EXPORT THE MODEL INTO TFLITE

In [17]:
CHATBOT_SAVED_MODEL = "exp_saved_model"

In [18]:
tf.saved_model.save(model, CHATBOT_SAVED_MODEL)

INFO:tensorflow:Assets written to: exp_saved_model/assets


In [19]:
%%bash -s $RPS_SAVED_MODEL
saved_model_cli show --dir $1 --tag_set serve --signature_def serving_default

Traceback (most recent call last):
  File "/usr/local/bin/saved_model_cli", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/tools/saved_model_cli.py", line 1260, in main
    args.func(args)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/tools/saved_model_cli.py", line 745, in show
    _show_inputs_outputs(args.dir, args.tag_set, args.signature_def)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/tools/saved_model_cli.py", line 152, in _show_inputs_outputs
    tag_set)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/tools/saved_model_utils.py", line 113, in get_meta_graph_def
    saved_model = read_saved_model(saved_model_dir)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/tools/saved_model_utils.py", line 51, in read_saved_model
    raise IOError("SavedModel file does not exist at: %s" % saved_model_dir)
OSError: SavedModel file does not exist at: $RPS_SAVED_M

In [20]:
loaded = tf.saved_model.load(CHATBOT_SAVED_MODEL)

In [21]:
print(list(loaded.signatures.keys()))
infer = loaded.signatures["serving_default"]
print(infer.structured_input_signature)
print(infer.structured_outputs)

['serving_default']
((), {'dense_input': TensorSpec(shape=(None, 45), dtype=tf.float32, name='dense_input')})
{'dense_2': TensorSpec(shape=(None, 6), dtype=tf.float32, name='dense_2')}


# CONVERT USING TFLITE CONVERTER

In [22]:
converter = tf.lite.TFLiteConverter.from_saved_model(CHATBOT_SAVED_MODEL)

In [23]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]

POST TRAINING INTEGER QUANTIZATION

In [ ]:
#def representative_data_gen():
    #for input_value, _ in test_batches.take(100):
        #yield [input_value]

In [ ]:
#converter.representative_dataset = representative_data_gen

FULL INTEGER QUANTIZATION

In [ ]:
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

CONVERT THE MODEL TO TFLITE

In [24]:
converter = tf.lite.TFLiteConverter.from_saved_model(CHATBOT_SAVED_MODEL)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()

In [25]:
tflite_model_file = 'converted_model.tflite'

with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

TES CHATBOT